Imprimo la hubicacion de la variable de entorno JAVA_HOME

Debe ser JAVA 64bits y la ruta no debe tener espacios ( Windows )

In [2]:
echo %JAVA_HOME%

C:\Progra~1\Java\jre1.8.0_271


Se configura pyspark para utilizar 20GB de RAM

In [3]:
import os
import pyspark

memory = '20g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

sc = pyspark.SparkContext('local[*]')

In [4]:
from pyspark.sql.types import StructType, DateType, IntegerType, StringType, DoubleType, BooleanType
from pyspark.sql.functions import datediff

In [5]:
sqlContext = pyspark.SQLContext(sc)

Date – The date of the file in yyyy-mm-dd format.

Serial Number – The manufacturer-assigned serial number of the drive.

Model – The manufacturer-assigned model number of the drive.

Capacity – The drive capacity in bytes.

Failure – Contains a “0” if the drive is OK. Contains a “1” if this is the last day the drive was operational before failing.


In [6]:
schema = StructType() \
      .add("Date", DateType(),True) \
      .add("Serial_Number",StringType(),True) \
      .add("Model",StringType(),True) \
      .add("capacity_bytes",DoubleType(),True) \
      .add("Failure",IntegerType(),True)

In [7]:
dia = sqlContext.read.options(header='True', delimiter=',') \
        .schema(schema) \
        .csv("../bigdata/**")

In [8]:
dia.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Serial_Number: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- capacity_bytes: double (nullable = true)
 |-- Failure: integer (nullable = true)



In [9]:
dia.registerTempTable("diskData")

In [10]:
sqlContext.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        | diskdata|       true|
+--------+---------+-----------+



Total de discos Discos del Datacenter con su fecha de Puesta en funcionamiento y dia de retiro

In [ ]:
df_AllDisksDayInDayOut = sqlContext.sql('Select Serial_Number, Model, Min(Date) as DayIn, Max(Date) as DayOut, Count(1) as Count from diskData group by Serial_Number, Model order by Model')

In [ ]:
df_AllDisksDayInDayOut.createOrReplaceTempView('tbl_AllDisksDayInDayOut')

Se agrega nueva tabla tbl_AllDisksDayInDayOut

In [ ]:
sqlContext.sql('show tables').show()

Esquema de tabla tbl_AllDisksDayInDayOut

In [ ]:
df_AllDisksDayInDayOut.printSchema()

Cantidad de discos por modelo que existen.

In [ ]:
sqlContext.sql('Select Model, Count(1) as Count from tbl_AllDisksDayInDayOut group by Model order by Count desc').show()

Listar la cantidad promedio de días que un dura un disco duro en operacion según su modelo.

In [ ]:
sqlContext.sql('Select CAST(AVG(DATEDIFF(DayOut, DayIn)) AS DECIMAL(10,2)) as Days, Model from tbl_AllDisksDayInDayOut Group by Model order by Days desc').show()

In [ ]:
sqlContext.sql('Select Count(1) as Cuenta, Model from diskData group by Model, Serial_Number order by Cuenta desc').show()

In [ ]:
sqlContext.sql('Select Count(1) as Cuenta, Model from diskData group by Model, Serial_Number order by Cuenta desc')\
          .repartition(1)\
          .write.format("parquet")\
          .save("output/parquet/transacciones", mode="OVERWRITE")

In [ ]:
df_averagedia.createOrReplaceTempView('tblAverageDia')

Cantidad de GB por modelo de disco duro

In [15]:
sqlContext.sql('select distinct first(Serial_Number) as SN, Model, CAST(first(capacity_bytes)/1073741824 AS DECIMAL(10,2)) as GB from diskData group by model').show()

+---------------+--------------------+--------+
|             SN|               Model|      GB|
+---------------+--------------------+--------+
|       S2X67QW1|         ST9250315AS|  232.89|
|       Z305B2QN|         ST4000DM000| 3726.02|
|WD-WCAWZ1087625|        WDC WD30EZRX| 2794.52|
|WD-WCAS85746595|      WDC WD5000AAJS|  465.76|
|       ZJV0XJQ4|       ST12000NM0007|11176.00|
|       W0Q6JBPX|          ST320LT007|  298.09|
|       ZA179XS0|         ST8000DM005| 7452.04|
|       9QM04GKZ|         ST3500320AS|  465.76|
|      Z79KT7B6T| TOSHIBA MQ01ABF050M|  465.76|
|WD-WCASY6801015|      WDC WD5002ABYS|  465.76|
|WD-WX71E31HV321|      WDC WD2500BEVT|  232.89|
| MJ0351YNGA723A|Hitachi HDS5C3030...| 2794.52|
|       ZA16NQJR|        ST8000NM0055| 7452.04|
|       W1E1TFP7|         ST2000DM001| 1863.02|
|       7M200214|Seagate BarraCuda...| 1863.02|
|   20H0A0CXF97G| TOSHIBA MG07ACA14TA|13039.00|
|WD-WX31DB48X22V|        WDC WD60EFRX| 5589.03|
|       5YD6PRD7|         ST1500DL001| 1

Top 5 modeos de discos más confiables por año

In [ ]:
sqlContext.sql('Select CAST(AVG(Diff) AS DECIMAL(10,2)) as Days, Model from (Select DATEDIFF(Max(Date), Min(Date)) as Diff, Serial_Number, Model from diskData group by Serial_Number, Model order by Model) As TableB Group by Model order by Days desc').show()

Top 5 modeos de discos más confiables por año

In [ ]:
sqlContext.sql('select DISTINCT count(1) as Cuenta, Model from diskData where group by Model, Serial_Number order by Cuenta desc').show(150)

In [ ]:
sqlContext.sql('select Serial_Number, count(1) from diskData group by Serial_Number order by count(1) desc').show(150)

In [ ]:
sqlContext.sql('select Date, Serial_Number, Model, Capacity, Failure from diskData where Failure = 1 and month(Date) = 2 and year(Date)=2020').show()

In [ ]:
selectedData = dia.select("Date", "Serial_Number")

In [ ]:
selectedData.write.csv('test2.csv')